In [13]:
import tkinter as tk
from tkinter import ttk
from fractions import Fraction
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

# Odds Conversion and Calculation Functions (as defined earlier)
# ... (include the conversion and calculation functions here)

def enhanced_calculate_arbitrage(odds1=None, odds2=None, stake1=None, odds1_type='decimal', odds2_type='decimal'):
    """
    Enhanced function to calculate arbitrage opportunities and other related metrics.
    
    Parameters:
    - odds1: float or str, odds for the first outcome (optional)
    - odds2: float or str, odds for the second outcome (optional)
    - stake1: float, stake for the first outcome (optional)
    - odds1_type: str, type of odds for the first outcome ('decimal', 'fractional', 'moneyline')
    - odds2_type: str, type of odds for the second outcome ('decimal', 'fractional', 'moneyline')
    
    Returns:
    - A dictionary containing various metrics
    """
    
    # Initialize variables
    result = {
        "threshold_odds2": None,
        "arbitrage_percentage": None,
        "is_arbitrage": False,
        "stake2": None,
        "total_stake": None,
        "total_payout": None,
        "profit": None,
        "profit_percentage": None
    }
    
    # Convert odds to decimal format
    if odds1 and odds1_type != 'decimal':
        if odds1_type == 'fractional':
            odds1 = fractional_to_decimal(odds1)
        elif odds1_type == 'moneyline':
            odds1 = moneyline_to_decimal(odds1)
    
    if odds2 and odds2_type != 'decimal':
        if odds2_type == 'fractional':
            odds2 = fractional_to_decimal(odds2)
        elif odds2_type == 'moneyline':
            odds2 = moneyline_to_decimal(odds2)
    
    # Calculate threshold odds for Outcome 2 if odds1 is provided
    if odds1:
        result["threshold_odds2"] = 1 / ((1 / odds1) - 0.01)
    
    # If both odds are provided, perform arbitrage calculations
    if odds1 and odds2:
        result["arbitrage_percentage"] = (1 / odds1 + 1 / odds2) * 100
        result["is_arbitrage"] = result["arbitrage_percentage"] < 100
    
    # If stake for Outcome 1 is provided and both odds are available, calculate the rest
    if stake1 and odds1 and odds2:
        # Calculate the stake for Outcome 2 that would ensure a $1 profit
        # The formula is derived from the equation: profit = payout1 - total_stake = payout2 - total_stake
        # Given profit = $1 and total_stake = stake1 + stake2, we find stake2
        result["stake2"] = ((stake1 * odds1) - 1) - stake1

        # Calculate the total stake and total payout
        result["total_stake"] = stake1 + result["stake2"]
        result["total_payout"] = stake1 * odds1  # Payout will be the same for either outcome
        
        # Calculate the profit and profit percentage
        result["profit"] = result["total_payout"] - result["total_stake"]
        result["profit_percentage"] = (result["profit"] / result["total_stake"]) * 100
    
    return result


def decimal_to_fractional(decimal_odds):
    """
    Convert Decimal Odds to Fractional Odds.
    """
    fractional_odds = Fraction(decimal_odds - 1).limit_denominator()
    return f"{fractional_odds.numerator}/{fractional_odds.denominator}"

def decimal_to_moneyline(decimal_odds):
    """
    Convert Decimal Odds to Moneyline (American) Odds.
    """
    if decimal_odds > 2:
        return int((decimal_odds - 1) * 100)
    else:
        return int(-100 / (decimal_odds - 1))

def fractional_to_decimal(fractional_odds):
    """
    Convert Fractional Odds to Decimal Odds.
    """
    numerator, denominator = map(int, fractional_odds.split('/'))
    return Fraction(numerator, denominator) + 1

def fractional_to_moneyline(fractional_odds):
    """
    Convert Fractional Odds to Moneyline (American) Odds.
    """
    decimal_odds = fractional_to_decimal(fractional_odds)
    return decimal_to_moneyline(decimal_odds)

def moneyline_to_decimal(moneyline_odds):
    """
    Convert Moneyline (American) Odds to Decimal Odds.
    """
    if moneyline_odds > 0:
        return (moneyline_odds / 100) + 1
    else:
        return (-100 / moneyline_odds) + 1

def moneyline_to_fractional(moneyline_odds):
    """
    Convert Moneyline (American) Odds to Fractional Odds.
    """
    decimal_odds = moneyline_to_decimal(moneyline_odds)
    return decimal_to_fractional(decimal_odds)






In [15]:
def autofill(*args):
    try:
        odds1 = float(odds1_var.get())
    except ValueError:
        return

    # Set stake1 to a value that will ensure $1 profit
    stake1 = 101  # Adjust this based on your specific logic
    stake1_var.set(stake1)
    
    # Calculate threshold odds2 that would ensure $1 profit
    threshold_odds2 = 102 / stake1  # Adjust this based on your specific logic
    odds2_var.set(round(threshold_odds2, 2))
    
    result = enhanced_calculate_arbitrage(odds1, threshold_odds2, stake1, 'moneyline', 'moneyline')
    update_result_labels(result)

def calculate():
    try:
        odds1 = float(odds1_var.get())
        odds2 = float(odds2_var.get())
        stake1 = float(stake1_var.get())
    except ValueError:
        messagebox.showwarning("Invalid Input", "Please enter valid numbers.")
        return
    
    result = enhanced_calculate_arbitrage(odds1, odds2, stake1, 'moneyline', 'moneyline')
    update_result_labels(result)

def update_result_labels(result):
    stake2_var.set(f"{result['stake2']:.2f}")
    total_stake_var.set(f"{result['total_stake']:.2f}")
    total_payout_var.set(f"{result['total_payout']:.2f}")
    profit_var.set(f"{result['profit']:.2f}")
    profit_percentage_var.set(f"{result['profit_percentage']:.2f}%")

root = tk.Tk()
root.title("Arbitrage & Hedge Bet Calculator")
root.geometry("500x500")

# Create Tkinter variables
odds1_var = tk.StringVar()
odds2_var = tk.StringVar()
stake1_var = tk.StringVar()
stake2_var = tk.StringVar()
total_stake_var = tk.StringVar()
total_payout_var = tk.StringVar()
profit_var = tk.StringVar()
profit_percentage_var = tk.StringVar()

# Add trace to the Tkinter variables
odds1_var.trace_add("write", autofill)

# Odds
odds1_label = ttk.Label(root, text="Odds 1:")
odds1_label.grid(row=0, column=0)
odds1_entry = ttk.Entry(root, textvariable=odds1_var)
odds1_entry.grid(row=0, column=1)

odds2_label = ttk.Label(root, text="Odds 2:")
odds2_label.grid(row=0, column=2)
odds2_entry = ttk.Entry(root, textvariable=odds2_var)
odds2_entry.grid(row=0, column=3)

# Stakes
stake1_label = ttk.Label(root, text="Stake 1:")
stake1_label.grid(row=1, column=0)
stake1_entry = ttk.Entry(root, textvariable=stake1_var)
stake1_entry.grid(row=1, column=1)

stake2_label = ttk.Label(root, text="Stake 2:")
stake2_label.grid(row=1, column=2)
stake2_entry = ttk.Label(root, textvariable=stake2_var)
stake2_entry.grid(row=1, column=3)

# Payouts (you can add code to handle this part)

# Totals
total_stake_label = ttk.Label(root, text="Total Stake:")
total_stake_label.grid(row=2, column=0)
total_stake_entry = ttk.Label(root, textvariable=total_stake_var)
total_stake_entry.grid(row=2, column=1)

total_payout_label = ttk.Label(root, text="Total Payout:")
total_payout_label.grid(row=3, column=0)
total_payout_entry = ttk.Label(root, textvariable=total_payout_var)
total_payout_entry.grid(row=3, column=1)

profit_label = ttk.Label(root, text="Profit:")
profit_label.grid(row=4, column=0)
profit_entry = ttk.Label(root, textvariable=profit_var)
profit_entry.grid(row=4, column=1)

profit_percentage_label = ttk.Label(root, text="Profit %:")
profit_percentage_label.grid(row=5, column=0)
profit_percentage_entry = ttk.Label(root, textvariable=profit_percentage_var)
profit_percentage_entry.grid(row=5, column=1)

# Add a Calculate button
calculate_button = ttk.Button(root, text="Calculate", command=calculate)
calculate_button.grid(row=6, column=0, columnspan=4)

root.mainloop()